## Indian School Education Statistics
https://www.kaggle.com/datasets/vidyapb/indian-school-education-statistics

Данный проект имеет задачу прогнозирования валового охвата в 2015-2016 годах с использованием данных сессии 2012-2015 годов, таких как процент отсева, наличие воды и компьютерного оборудования. Решение задачи основано на проблеме классификации, где общее количество учащихся разделено на две группы.

Ниже приведены этапы, выполненные в данном проекте:

1. Импортируются необходимые библиотеки и загружаются данные из CSV-файлов.
2. Выполняется предварительная обработка данных, включая исправление имен штатов и заполнение пропущенных значений с помощью метода SimpleImputer.
3. Данные о проценте отсева обрабатываются для создания средних значений для каждого типа учащихся и уровня образования (первичное, среднее, старшее и высшее).
4. Аналогичные шаги выполняются для данных о наличии воды, компьютерного оборудования, электричества и туалетов для мальчиков и девочек.
5. Создаются наборы данных для каждого уровня образования, объединяя средние значения различных параметров.
6. Подготавливаются наборы данных для оценки, включая данные за 2015-2016 годы.
7. Строятся модели машинного обучения для каждого уровня образования, в данном случае Random Forest Classifier.
8. Оценивается производительность моделей с помощью различных метрик, таких как accuracy score, roc-auc score, precision score, log loss, f1 score и mean squared error.

In [1]:
#нужно для того, что бы скрывать ворнинги 
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer #нужно для заполнения пропусков
from sklearn.model_selection import train_test_split #для создания подвыборок
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold #кроссвалидация
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, log_loss, f1_score, mean_squared_error